In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("github_repos_wildcard")

In [2]:
repo_url = f"https://{token}@github.com/gaserSami/panther.git"
branch = "graphing"

In [3]:
!git clone -b {branch} {repo_url}

Cloning into 'panther'...
remote: Enumerating objects: 625, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 625 (delta 105), reused 75 (delta 69), pack-reused 502 (from 1)
Receiving objects: 100% (625/625), 27.49 MiB | 19.03 MiB/s, done.
Resolving deltas: 100% (351/351), done.


In [4]:
!mv panther Panther

In [5]:
# !pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu118

In [6]:
# import torch
# print(torch.__version__)
# import triton
# print(triton.__version__)

In [7]:
# !export LC_ALL="en_US.UTF-8"
# !export LD_LIBRARY_PATH="/usr/lib64-nvidia"
# !export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
# !ldconfig /usr/lib64-nvidia

In [8]:
%%writefile /kaggle/working/Panther/pawX/setup.py
from setuptools import setup
from torch.utils.cpp_extension import BuildExtension, CUDAExtension

setup(
    name="pawX",
    ext_modules=[
        CUDAExtension(
            name="pawX",
            sources=[
                "skops.cpp",
                "bindings.cpp",
                "linear.cpp",
                "linear_cuda.cu",
                "cqrrpt.cpp",
                "rsvd.cpp",
                "attention.cpp",
            ],
            # Use system includes and libraries
            include_dirs=["/usr/include/x86_64-linux-gnu"],
            library_dirs=[],
            libraries=["openblas"],
            extra_compile_args={"cxx": ["-O2", "-fopenmp"], "nvcc": ["-O2"]},
            extra_link_args=["-llapacke", "-lopenblas"]
        )
    ],
    cmdclass={"build_ext": BuildExtension},
)

Overwriting /kaggle/working/Panther/pawX/setup.py


In [9]:
# %%writefile /kaggle/working/Panther/panther/nn/linear.py
# import math
# from typing import Any, Tuple

# import torch
# import torch.nn as nn
# from torch.autograd import Function
# from torch.nn import init

# from panther.sketch import scaled_sign_sketch as gen_U
# from pawX import sketched_linear_backward, sketched_linear_forward


# class SketchedLinearFunction(Function):
#     # Note that forward, setup_context, and backward are @staticmethods
#     @staticmethod
#     def forward(
#         ctx,
#         input: torch.Tensor,
#         S1s: torch.Tensor,
#         S2s: torch.Tensor,
#         U1s: torch.Tensor,
#         U2s: torch.Tensor,
#         bias: torch.Tensor,
#         mode: int,
#     ):
#         return sketched_linear_forward(input, S1s, S2s, U1s, U2s, bias, mode)

#     @staticmethod
#     # inputs is a Tuple of all of the inputs passed to forward.
#     # output is the output of the forward().
#     def setup_context(ctx: Any, inputs: Tuple[Any, ...], output: Any):
#         input, S1s, S2s, U1s, U2s, bias, mode = inputs
#         ctx.save_for_backward(input, S1s, S2s, U1s, U2s, bias, mode)

#     @staticmethod
#     def backward(ctx: Any, *grad_output: Any) -> Any:
#         # dl/dS2_i = U1_i g h_in^T / 2 * l
#         # dl/dS1_i = g h_in^T U2_i^T / 2 * l
#         # dl/dh_in = 1/(2*l) * (sum_{i=1}^{l} (S1_i^T U1_i g) + sum_{i=1}^{l} (U2_i^T S2_i g))
#         # dl/db = g
#         input, S1s, S2s, U1s, U2s, _ = ctx.saved_tensors
#         grads = sketched_linear_backward(
#             grad_output=grad_output[0],
#             input=input,
#             S1s=S1s,
#             S2s=S2s,
#             U1s=U1s,
#             U2s=U2s,
#         )
#         return (
#             grads[0],  # h_in
#             grads[1],  # S1s
#             grads[2],  # S2s
#             None,  # U1s
#             None,  # U2s
#             grads[3],  # bias
#             None,
#         )


# class SKLinear(nn.Module):
#     __constants__ = ["in_features", "out_features", "num_terms", "low_rank"]
#     in_features: int
#     out_features: int
#     num_terms: int
#     low_rank: int
#     S1s: torch.Tensor
#     S2s: torch.Tensor
#     U1s: torch.Tensor
#     U2s: torch.Tensor

#     def __init__(
#         self,
#         in_features: int,
#         out_features: int,
#         num_terms: int,
#         low_rank: int,
#         mode: int,
#         W_init=None,
#         bias: bool = True,
#         dtype=None,
#         device=None,
#     ):
#         factory_kwargs = {"device": device, "dtype": dtype}
#         super(SKLinear, self).__init__()

#         # if (
#         #     2 * num_terms * low_rank * (out_features + in_features)
#         #     > out_features * in_features
#         # ):
#         #     raise ValueError(
#         #         "The number of parameters in the sketching layer is larger "
#         #         + "than the number of parameters in the fully connected layer."
#         #     )
#         self.mode = mode
#         self.num_terms = num_terms
#         self.low_rank = low_rank
#         self.out_features = out_features
#         self.in_features = in_features

#         # Register U1s and U2s as buffers since they are not learnable
#         self.register_buffer(
#             "U1s",
#             torch.stack(
#                 [
#                     gen_U(low_rank, out_features, **factory_kwargs)
#                     for _ in range(num_terms)
#                 ]
#             ),
#         )  # kxd1
#         self.register_buffer(
#             "U2s",
#             torch.stack(
#                 [
#                     gen_U(low_rank, in_features, **factory_kwargs).T
#                     for _ in range(num_terms)
#                 ]
#             ),
#         )  # d2xk

#         # W is used to only initialize S
#         if W_init is None:
#             W = torch.empty(in_features, out_features, **factory_kwargs)
#             init.kaiming_uniform_(W, a=math.sqrt(5))
#         else:
#             W = W_init.T.detach().clone()

#         # S1s and S2s are precomputed but not updated in the backward pass
#         self.S1s = nn.Parameter(
#             torch.stack([torch.matmul(W, self.U1s[i].T) for i in range(num_terms)])
#         )  # d2xk
#         self.S2s = nn.Parameter(
#             torch.stack([torch.matmul(self.U2s[i].T, W) for i in range(num_terms)])
#         )  # kxd1

#         # Bias term initialized with a small standard deviation
#         if bias:
#             self.bias = nn.Parameter(torch.empty(out_features, **factory_kwargs))
#             fan_in, _ = init._calculate_fan_in_and_fan_out(W)
#             bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
#             init.uniform_(self.bias, -bound, bound)
#         else:
#             self.register_parameter("bias", None)

#     def forward(self, h_in):
#         return SketchedLinearFunction.apply(
#             h_in, self.S1s, self.S2s, self.U1s, self.U2s, self.bias, 0
#         )


# if __name__ == "__main__":
#     linear = SKLinear(
#         in_features=10,
#         out_features=10,
#         num_terms=1,
#         low_rank=1,
#         mode=0,
#         dtype=torch.float32,
#         device=torch.device("cuda:0"),
#     )
#     input = torch.randn(1, 10, dtype=torch.float32, device=torch.device("cuda:0"))
#     output = linear(input)
#     print(output)
#     print(output.shape)


In [10]:
!sudo apt-get install liblapacke-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  liblapacke libtmglib-dev libtmglib3
Suggested packages:
  liblapack-doc
The following NEW packages will be installed:
  liblapacke liblapacke-dev libtmglib-dev libtmglib3
0 upgraded, 4 newly installed, 0 to remove and 122 not upgraded.
Need to get 1,071 kB of archives.
After this operation, 12.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libtmglib3 amd64 3.10.0-2ubuntu1 [144 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblapacke amd64 3.10.0-2ubuntu1 [435 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libtmglib-dev amd64 3.10.0-2ubuntu1 [134 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblapacke-dev amd64 3.10.0-2ubuntu1 [358 kB]
Fetched 1,071 kB in 1s (969 kB/s)       
debconf: unable to initialize frontend: Dialog
debconf: (No usable di

In [11]:
!cd /kaggle/working/Panther/pawX; python setup.py install
!cd /kaggle/working/Panther/pawX; pip install --no-build-isolation -e .

/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/py

In [12]:
import os

In [13]:
os.chdir("/kaggle/working/Panther/")

In [14]:
!pwd

/kaggle/working/Panther


In [ ]:
import time
import numpy as np
import torch
import torch._dynamo
import torch._inductor.config as config
import itertools
import pandas as pd

# Configure torch
config.max_autotune_gemm = False
torch._dynamo.config.cache_size_limit = 1024
torch._dynamo.config.accumulated_cache_size_limit = 1024

def is_valid_params(in_features, out_features, num_terms, low_rank):
    """
    Check if parameter combination is valid:
    A combination is invalid if 2 * num_terms * low_rank * (out_features + in_features) >= out_features * in_features
    """
    return 2 * num_terms * low_rank * (out_features + in_features) < out_features * in_features

class BenchmarkParams:
    def __init__(self, 
                 in_features=256, 
                 out_features=256,
                 num_terms=3,
                 low_rank=8,
                 batch_size=64, 
                 num_runs=200, 
                 warmup=15, 
                 device=torch.device("cuda"),
                 dtype=torch.float32
                ,mode=0):
        self.in_features = in_features
        self.out_features = out_features
        self.num_terms = num_terms
        self.low_rank = low_rank
        self.batch_size = batch_size
        self.num_runs = num_runs
        self.warmup = warmup
        self.device = device
        self.dtype = dtype
        self.mode = mode

def benchmark_model(model, x, model_name, params):
    """
    Generic benchmarking function for any PyTorch model.
    
    Args:
        model: The PyTorch model to benchmark
        x: Input tensor
        model_name: Name of the model for logging
        params: Benchmark parameters
    
    Returns:
        Dictionary with benchmark results
    """
    # Compile the model
    # model_compiled = torch.compile(
    #     model,
    #     backend="inductor",
    #     fullgraph=True,
    #     dynamic=False
    # )
    model_compiled = model
    
    # Benchmark forward pass
    print(f"\n=== {model_name} FORWARD PASS BENCHMARK ===")
    
    # Warmup runs for forward pass
    model_compiled.eval()
    with torch.no_grad():
        for _ in range(params.warmup):
            _ = model_compiled(x)
    
    torch.cuda.synchronize()
    
    # Actual timed runs for forward
    forward_times = []
    with torch.no_grad():
        for _ in range(params.num_runs):
            torch.cuda.synchronize()
            start = time.perf_counter()
            _ = model_compiled(x)
            torch.cuda.synchronize()
            end = time.perf_counter()
            
            forward_times.append((end - start) * 1000)  # Convert to ms
    
    mean_forward = np.mean(forward_times)
    std_forward = np.std(forward_times)
    print(f"{model_name} forward: {mean_forward:.3f} ± {std_forward:.3f} ms")
    
    # Benchmark backward pass
    print(f"\n=== {model_name} BACKWARD PASS BENCHMARK ===")
    
    # Warmup runs for backward pass
    model_compiled.train()
    for _ in range(params.warmup):
        out = model_compiled(x)
        loss = out.sum()
        loss.backward()
        x.grad.zero_()
    
    torch.cuda.synchronize()
    
    # Actual timed runs for backward
    backward_times = []
    for _ in range(params.num_runs):
        out = model_compiled(x)
        loss = out.sum()
        
        torch.cuda.synchronize()
        start = time.perf_counter()
        loss.backward()
        torch.cuda.synchronize()
        end = time.perf_counter()
        
        backward_times.append((end - start) * 1000)  # Convert to ms
        x.grad.zero_()
    
    mean_backward = np.mean(backward_times)
    std_backward = np.std(backward_times)
    print(f"{model_name} backward: {mean_backward:.3f} ± {std_backward:.3f} ms")
    
    return {
        "forward": {
            "mean": mean_forward,
            "std": std_forward,
            "times": forward_times
        },
        "backward": {
            "mean": mean_backward,
            "std": std_backward,
            "times": backward_times
        }
    }

def benchmark_model_factory(model_factory, model_name, params):
    """
    Benchmark a model using a factory function.
    
    Args:
        model_factory: Function that creates the model
        model_name: Name of the model for logging
        params: Benchmark parameters
    
    Returns:
        Dictionary with benchmark results
    """
    # Create the model
    torch.manual_seed(42)
    model = model_factory(params)
    
    # Create input tensor for benchmarking
    x = torch.randn(params.batch_size, params.in_features, 
                  dtype=params.dtype, device=params.device, requires_grad=True)
    
    return benchmark_model(model, x, model_name, params)

if __name__ == "__main__":
    import torch.nn as nn
    from panther.nn import SKLinear
    
    # Parameter combinations to test
    ratios = [(1, 128), (128, 1), (1, 1), (2, 1), (1, 2)]
    base_sizes = [256, 512, 1024, 8192, 16384]
    num_terms_options = [1, 2, 3]
    low_rank_options = [10, 15, 20, 50, 100, 150]
    
    # ratios = [(1, 128)]
    # base_sizes = [256, ]
    # num_terms_options = [3]
    # low_rank_options = [15]
    
    # Define model factories
    def create_sklinear(p):
        return SKLinear(in_features=p.in_features, out_features=p.out_features, 
                       num_terms=p.num_terms, low_rank=p.low_rank, mode=p.mode,
                       dtype=p.dtype, device=p.device)
    
    models_to_benchmark = [
        (create_sklinear, "SKLinear")
    ]
    
    # Prepare data structure to store all results
    results_data = []
    
    # Iterate through all parameter combinations
    total_combinations = len(ratios) * len(base_sizes) * len(num_terms_options) * len(low_rank_options)
    current_combo = 0
    
    for ratio, base_size in itertools.product(ratios, base_sizes):
        ratio_in, ratio_out = ratio
        
        # Calculate actual dimensions based on ratio and base size
        if ratio_in == 1:
            in_features = base_size
            out_features = base_size * ratio_out
        else:
            out_features = base_size
            in_features = base_size * ratio_in
        
        for num_terms, low_rank in itertools.product(num_terms_options, low_rank_options):
            current_combo += 1
            print(f"\n\n{'='*20} COMBINATION {current_combo}/{total_combinations} {'='*20}")
            print(f"In features: {in_features}, Out features: {out_features}, Ratio: {ratio_in}:{ratio_out}")
            print(f"Base size: {base_size}, Num terms: {num_terms}, Low rank: {low_rank}")
            
            # Check if parameters are valid
            is_valid = is_valid_params(in_features, out_features, num_terms, low_rank)
            
            if not is_valid:
                print(f"INVALID COMBINATION: 2 * {num_terms} * {low_rank} * ({out_features} + {in_features}) >= {out_features} * {in_features}")
                print("Skipping benchmarks for this invalid combination")
                
                # Add invalid entry to results data
                for model_name in [m[1] for m in models_to_benchmark]:
                    results_data.append({
                        'model': model_name,
                        'in_features': in_features,
                        'out_features': out_features,
                        'ratio': f"{ratio_in}:{ratio_out}",
                        'base_size': base_size,
                        'num_terms': num_terms,
                        'low_rank': low_rank,
                        'forward_mean_ms': float('nan'),
                        'forward_std_ms': float('nan'),
                        'backward_mean_ms': float('nan'),
                        'backward_std_ms': float('nan'),
                        'is_valid': False,
                        'error': "Invalid parameter combination"
                    })
                continue
            
            # Create parameter object for this combination
            params = BenchmarkParams(
                in_features=in_features,
                out_features=out_features,
                num_terms=num_terms,
                low_rank=low_rank
            )

            
            all_results = {}
            for model_factory, model_name in models_to_benchmark:
                print(f"\n{'='*20} Benchmarking {model_name} {'='*20}")
                if True:
                    results = benchmark_model_factory(model_factory, model_name, params)
                    all_results[model_name] = results
                    
                    # Add result to our data collection
                    results_data.append({
                        'model': model_name,
                        'in_features': in_features,
                        'out_features': out_features,
                        'ratio': f"{ratio_in}:{ratio_out}",
                        'base_size': base_size,
                        'num_terms': num_terms,
                        'low_rank': low_rank,
                        'forward_mean_ms': results['forward']['mean'],
                        'forward_std_ms': results['forward']['std'],
                        'backward_mean_ms': results['backward']['mean'],
                        'backward_std_ms': results['backward']['std'],
                        'is_valid': True
                    })
                # except Exception as e:
                    print(f"Error benchmarking {model_name}: {e}")
                    # Add error entry to data
                    results_data.append({
                        'model': model_name,
                        'in_features': in_features,
                        'out_features': out_features,
                        'ratio': f"{ratio_in}:{ratio_out}",
                        'base_size': base_size,
                        'num_terms': num_terms,
                        'low_rank': low_rank,
                        'forward_mean_ms': float('nan'),
                        'forward_std_ms': float('nan'),
                        'backward_mean_ms': float('nan'),
                        'backward_std_ms': float('nan'),
                        'is_valid': True,
                        'error': str(e)
                    })
            
            # Print comparative summary for this combination
            if all_results:
                print("\n" + "="*60)
                print(f"{'='*20} SUMMARY FOR CURRENT COMBINATION {'='*20}")
                print("="*60)
                print(f"{'Model':<20} {'Forward (ms)':<25} {'Backward (ms)':<25}")
                print("-"*60)
                
                for model_name, results in all_results.items():
                    fwd = f"{results['forward']['mean']:.3f} ± {results['forward']['std']:.3f}"
                    bwd = f"{results['backward']['mean']:.3f} ± {results['backward']['std']:.3f}"
                    print(f"{model_name:<20} {fwd:<25} {bwd:<25}")
    
    # Create a DataFrame with all results
    df = pd.DataFrame(results_data)
    
    # Save results to CSV
    results_file = "benchmark_results.csv"
    df.to_csv(results_file, index=False)
    print(f"\nAll benchmark results saved to {results_file}")



==================== COMBINATION 1/450 ====================
In features: 256, Out features: 32768, Ratio: 1:128
Base size: 256, Num terms: 1, Low rank: 10

==================== Benchmarking SKLinear ====================

=== SKLinear FORWARD PASS BENCHMARK ===
Ideal Block Size (1D): 512
Resident Threads per SM: 1024 (out of 1024 available)
Ideal Block Size (1D): 512
Resident Threads per SM: 1024 (out of 1024 available)
Ideal Block Size (1D): 512
Resident Threads per SM: 1024 (out of 1024 available)
Ideal Block Size (1D): 512
Resident Threads per SM: 1024 (out of 1024 available)
Ideal Block Size (1D): 512
Resident Threads per SM: 1024 (out of 1024 available)
Ideal Block Size (1D): 512
Resident Threads per SM: 1024 (out of 1024 available)
Ideal Block Size (1D): 512
Resident Threads per SM: 1024 (out of 1024 available)
Ideal Block Size (1D): 512
Resident Threads per SM: 1024 (out of 1024 available)
Ideal Block Size (1D): 512
Resident Threads per SM: 1024 (out of 1024 available)
Ideal Bl

In [ ]:
os.chdir("/kaggle/working/Panther/tests")

In [ ]:
# !pytest